# Infrared Thermography (IRT) Reconstruction Demo (Iterative Diffusion Model)
This notebook demonstrates TV-regularized reconstruction for Infrared Thermography. It aims to reconstruct an **initial subsurface heat distribution** from surface temperature measurements observed over time. 
The `InfraredThermographyOperator` simulates heat diffusion by iteratively applying a Gaussian blur kernel to an initial heat map. The reconstruction then attempts to recover this initial heat map.

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Adjust path to import from reconlib 
import sys
# sys.path.append('../../../') # Adjust as needed

from reconlib.modalities.infrared_thermography.operators import InfraredThermographyOperator
from reconlib.modalities.infrared_thermography.reconstructors import tv_reconstruction_irt
from reconlib.modalities.infrared_thermography.utils import generate_irt_phantom, plot_irt_results

print(f"PyTorch version: {torch.__version__}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Setup Parameters and Phantom (Initial Heat Map)

In [ ]:
initial_heat_map_shape = (64, 64)  # (Ny, Nx) for the initial heat distribution
num_time_steps = 10                # Number of time frames for surface temperature observation
diffusion_sigma = 1.5              # Sigma for the Gaussian diffusion kernel in the operator

# Generate a phantom representing the initial heat distribution.
# 'hotspot' type from generate_irt_phantom is suitable here.
true_initial_heat_map = generate_irt_phantom(
    image_shape=initial_heat_map_shape, 
    num_defects=3, 
    defect_type='hotspot', # Simulates localized initial heat sources
    device=device
)

plt.figure(figsize=(6,6))
plt.imshow(true_initial_heat_map.cpu().numpy(), cmap='hot')
plt.title('True Initial Heat Map')
plt.xlabel('X (pixels)')
plt.ylabel('Y (pixels)')
plt.colorbar(label='Initial Heat Intensity')
plt.show()

## 2. Initialize Operator and Simulate Surface Temperature Evolution
The IRT operator simulates how the initial heat diffuses over time, leading to a sequence of surface temperature maps.

In [ ]:
irt_operator = InfraredThermographyOperator(
    image_shape=initial_heat_map_shape,
    time_steps=num_time_steps,
    diffusion_kernel_sigma=diffusion_sigma,
    device=device
)

# Simulate surface temperature data using the forward operator
y_surface_temps = irt_operator.op(true_initial_heat_map)

print(f"Simulated surface temperature data shape: {y_surface_temps.shape}")

# Visualize a few time frames of the simulated surface temperature
fig, axes = plt.subplots(1, min(num_time_steps, 4), figsize=(min(num_time_steps,4)*4, 4))
if num_time_steps == 0: # Handle case of no time steps, though op expects >0
    print("No time steps to display.")
elif min(num_time_steps,4) == 1 and num_time_steps >=1:
    axes = [axes]
    im = axes[0].imshow(y_surface_temps[0].cpu().numpy(), cmap='inferno')
    axes[0].set_title(f'Surface Temp (t=0)')
    axes[0].set_xlabel('X'); axes[0].set_ylabel('Y')
    fig.colorbar(im, ax=axes[0], fraction=0.046, pad=0.04)
else: # Multiple frames to show
    for i, ax_idx in enumerate(np.linspace(0, num_time_steps-1, min(num_time_steps, 4)).astype(int)):
        im = axes[i].imshow(y_surface_temps[ax_idx].cpu().numpy(), cmap='inferno')
        axes[i].set_title(f'Surface Temp (t={ax_idx})')
        axes[i].set_xlabel('X'); axes[i].set_ylabel('Y')
        fig.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

plt.suptitle('Simulated Surface Temperatures (Iterative Diffusion Model)')
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 3. Perform Reconstruction
Using Total Variation (TV) regularization with Proximal Gradient to reconstruct the initial heat map.

In [ ]:
lambda_tv_irt = 0.0005     # TV regularization strength (may need tuning)
iterations_irt = 75        # Number of proximal gradient iterations
step_size_irt = 1e-3       # Step size for proximal gradient (CRITICAL, adjust based on data scaling and lambda)
tv_prox_iters_irt = 5      # Iterations for TV prox

# Perform reconstruction
reconstructed_initial_heat = tv_reconstruction_irt(
    y_surface_temperature_sequence=y_surface_temps,
    irt_operator=irt_operator,
    lambda_tv=lambda_tv_irt,
    iterations=iterations_irt,
    step_size=step_size_irt,
    tv_prox_iterations=tv_prox_iters_irt,
    is_3d_tv=False, # Current operator and phantom are 2D
    verbose=True
)

print(f"Reconstructed initial heat map shape: {reconstructed_initial_heat.shape}")

## 4. Display Results

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

im1 = axes[0].imshow(true_initial_heat_map.cpu().numpy(), cmap='hot')
axes[0].set_title('True Initial Heat Map')
axes[0].set_xlabel('X (pixels)')
axes[0].set_ylabel('Y (pixels)')
fig.colorbar(im1, ax=axes[0], fraction=0.046, pad=0.04)

im2 = axes[1].imshow(reconstructed_initial_heat.cpu().numpy(), cmap='hot')
axes[1].set_title(f'Reconstructed Initial Heat (TV, {iterations_irt} iters)')
axes[1].set_xlabel('X (pixels)')
axes[1].set_ylabel('Y (pixels)')
fig.colorbar(im2, ax=axes[1], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

# Using the utility plot function (currently a placeholder itself)
plot_irt_results(
    subsurface_map_true=true_initial_heat_map,
    subsurface_map_recon=reconstructed_initial_heat,
    surface_temp_sequence=y_surface_temps,
    time_slice_to_display=num_time_steps-1 # Show last time frame of surface temp for context
)

## 5. Further Considerations
The current `InfraredThermographyOperator` uses a simplified iterative diffusion model.
1. **Realistic Heat Transfer Physics**: For accurate IRT modeling, solving the heat equation (e.g., using Finite Difference or Finite Element methods) with appropriate material properties (thermal conductivity, diffusivity, density) and boundary conditions is essential.
2. **Adjoint State Method**: For complex heat equation solvers, the adjoint state method is often used to correctly compute the gradient for iterative reconstruction, ensuring the `op_adj` is the true adjoint of `op`.
3. **Active vs. Passive Thermography**: This demo is closer to a passive scenario (observing diffusion of initial heat). Active thermography (pulsed, lock-in) involves modeling external heat sources and their interaction, which would require different operator implementations.
4. **Calibration & Emissivity**: Real IRT data requires calibration to temperature and corrections for surface emissivity.
5. **Noise Models**: Thermal noise and detector noise in IR cameras can be complex. More accurate noise models in simulation and data fidelity terms in reconstruction can improve results.